In [221]:
# Import libraries
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import os
import json

In [222]:
model = gp.Model("model1")
x = model.addVar(name="x", vtype=GRB.CONTINUOUS)
y = model.addVar(name="y", vtype=GRB.CONTINUOUS)
z = model.addVar(name="z", vtype=GRB.CONTINUOUS)

model.addConstr(x + y + z <= 10, name="c1")

model.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

model.optimize()

for v in model.getVars():
    print(f"{v.varName}: {v.x}")

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x0e99420c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 1 rows and 3 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.000000000e+01
x: 0.0
y: 0.0
z: 10.0


In [223]:
#Open jason file to load data
with open(r"C:\Users\thomv\Documents\ULB\MA2\Combinatorial optimization\Project\instances\example.json", "r") as file:
    data = json.load(file)


In [224]:
#print all attributes of the json file
for key, value in data.items():
    print(f"{key}: {value}")

Blocks: ['Block 001']
Sessions: ['Session 001', 'Session 002', 'Session 003']
Sessions_b: {'Block 001': ['Session 001', 'Session 002', 'Session 003']}
Interpreters: ['Interpreter 001', 'Interpreter 002', 'Interpreter 003', 'Interpreter 004', 'Interpreter 005']
Languages: ['Dutch', 'English', 'French', 'German', 'Italian', 'Spanish']
Languages_s: {'Session 001': ['English', 'French', 'Spanish'], 'Session 002': ['German', 'Spanish'], 'Session 003': ['English', 'Spanish']}
Languages_i: {'Interpreter 001': ['English', 'Spanish'], 'Interpreter 002': ['French', 'Spanish'], 'Interpreter 003': ['Dutch', 'German'], 'Interpreter 004': ['Dutch', 'Spanish'], 'Interpreter 005': ['French', 'Italian', 'Spanish']}


In [225]:
#Unpack data from json file
Blocks = data["Blocks"]
Sessions = data["Sessions"]
Sessions_blocks = data["Sessions_b"]
Interpreters = data["Interpreters"]
Languages = data["Languages"]
Languages_sessions = data["Languages_s"]
Languages_interpreters = data["Languages_i"]

In [226]:
print("Blocks:", Blocks)
print("sessions", Sessions)
print("Languages_sessions:", Languages_sessions)
print("Languages_interpreters:", Languages_interpreters)
print("Interpreters:", Interpreters)

Blocks: ['Block 001']
sessions ['Session 001', 'Session 002', 'Session 003']
Languages_sessions: {'Session 001': ['English', 'French', 'Spanish'], 'Session 002': ['German', 'Spanish'], 'Session 003': ['English', 'Spanish']}
Languages_interpreters: {'Interpreter 001': ['English', 'Spanish'], 'Interpreter 002': ['French', 'Spanish'], 'Interpreter 003': ['Dutch', 'German'], 'Interpreter 004': ['Dutch', 'Spanish'], 'Interpreter 005': ['French', 'Italian', 'Spanish']}
Interpreters: ['Interpreter 001', 'Interpreter 002', 'Interpreter 003', 'Interpreter 004', 'Interpreter 005']


In [227]:
def Model(data, OBJ_FUNC=1):
    model = gp.Model("model1")
    
    #unpack data
    Blocks = data["Blocks"]
    Sessions = data["Sessions"]
    Sessions_blocks = data["Sessions_b"]
    Interpreters = data["Interpreters"]
    Languages = data["Languages"]
    Languages_sessions = data["Languages_s"]
    Languages_interpreters = data["Languages_i"]

    #Generate the pairwise combinations of languages
    Pairwise_languages_sessions = Languages_sessions.copy()
    for s in Sessions:
        Pairwise_languages_sessions[s] = [(l1, l2) for i, l1 in enumerate(Languages_sessions[s]) 
                                          for l2 in Languages_sessions[s][i+1:]]
    print("Pairwise languages sessions:", Pairwise_languages_sessions)

    # Create variables
    #x[b, s, language, interpreter] = 1 if block b is assigned to session s with language l and interpreter i
    assign = {}
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l in Pairwise_languages_sessions[s]:
                for i in Interpreters:
                    assign[b, s, l, i] = model.addVar(vtype=GRB.BINARY, name=f"x_{b}_{s}_{l}_{i}")

    session_covered = {}
    for s in Sessions:
        #session_covered[s] = 1 if session s is covered by at least one interpreter
        session_covered[s] = model.addVar(vtype=GRB.BINARY, name=f"session_covered_{s}")
    
    # Add constraints
    #Interpreter must speak the language
    for b in Blocks:
        for s in Sessions_blocks[b]:
            #change by pairwise language
            for l in Pairwise_languages_sessions[s]:
                l1, l2 = l
                for i in Interpreters:
                    if l1 not in Languages_interpreters[i] or l2 not in Languages_interpreters[i]:
                        model.addConstr(assign[b, s, l, i] == 0, name=f"lang_{b}_{s}_{l}_{i}")
    
    #One interpreter can translate only one session at a time
    for b in Blocks:
        for i in Interpreters:
            model.addConstr(gp.quicksum(assign[b, s, l, i] for s in Sessions_blocks[b] for l in Pairwise_languages_sessions[s]) <= 1, 
                            name=f"interpreter_{b}_{i}")
    
    # Each session can have only one interpreter per language
    for s in Sessions:
        for l in Pairwise_languages_sessions[s]:
            model.addConstr(gp.quicksum(assign[b, s, l, i] for b in Blocks for i in Interpreters) <= 1, 
                            name=f"session_{s}_{l}")
            
    #Session covered if all languages are covered
    for s in Sessions:
        model.addConstr(session_covered[s] <= gp.quicksum(assign[b, s, l, i] for b in Blocks
                                                          for l in Pairwise_languages_sessions[s] 
                                                          for i in Interpreters) / len(Pairwise_languages_sessions[s]), 
                                                          name=f"session_covered_{s}")

    # Set objective
    #Maximize the number of languages translated
    #OBJ1
    if OBJ_FUNC == 1:
        model.setObjective(gp.quicksum(assign[b, s, l, i] for b in Blocks for s in Sessions_blocks[b] 
                                        for l in Pairwise_languages_sessions[s] for i in Interpreters), GRB.MAXIMIZE)
    #
    #Max the number of sessions translated
    if OBJ_FUNC == 2:
        model.setObjective(gp.quicksum(session_covered[s] for b in Blocks for s in Sessions_blocks[b]), GRB.MAXIMIZE)
    
    return model

In [228]:
model = Model(data)
model.optimize()
# Print the results
for v in model.getVars():
    if (v.x ==1):
        print(f"{v.varName}: {v.x}")

Pairwise languages sessions: {'Session 001': [('English', 'French'), ('English', 'Spanish'), ('French', 'Spanish')], 'Session 002': [('German', 'Spanish')], 'Session 003': [('English', 'Spanish')]}
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 34 rows, 28 columns and 99 nonzeros
Model fingerprint: 0x2391d37b
Variable types: 0 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 34 rows and 28 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available 

In [229]:
model = Model(data, 2)
model.optimize()
# Print the results
for v in model.getVars():
    if (v.x ==1):
        print(f"{v.varName}: {v.x}")

Pairwise languages sessions: {'Session 001': [('English', 'French'), ('English', 'Spanish'), ('French', 'Spanish')], 'Session 002': [('German', 'Spanish')], 'Session 003': [('English', 'Spanish')]}
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 34 rows, 28 columns and 99 nonzeros
Model fingerprint: 0x3df7858a
Variable types: 0 continuous, 28 integer (28 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 34 rows and 28 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 12 available

In [239]:
def Model_2(data, OBJ_FUNC=1):
    model = gp.Model("model2")
    
    
    #unpack data
    Blocks = data["Blocks"]
    Sessions = data["Sessions"]
    Sessions_blocks = data["Sessions_b"]
    Interpreters = data["Interpreters"]
    Languages = data["Languages"]
    Languages_sessions = data["Languages_s"]
    Languages_interpreters = data["Languages_i"]

    #Generate the pairwise combinations of languages
    Pairwise_languages_sessions = Languages_sessions.copy()
    for s in Sessions:
        Pairwise_languages_sessions[s] = [(l1, l2) for i, l1 in enumerate(Languages_sessions[s]) 
                                          for l2 in Languages_sessions[s][i+1:]]
    print("Pairwise languages sessions:", Pairwise_languages_sessions)

    # Create variables
    #x[b, s, language, interpreter] = 1 if block b is assigned to session s with language l and interpreter i
    assign = {}
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l in Pairwise_languages_sessions[s]:
                for i in Interpreters:
                    assign[b, s, l, i] = model.addVar(vtype=GRB.BINARY, name=f"x_{b}_{s}_{l}_{i}")

    session_covered = {}
    for s in Sessions:
        #session_covered[s] = 1 if session s is covered by at least one interpreter
        session_covered[s] = model.addVar(vtype=GRB.BINARY, name=f"session_covered_{s}")
    
    # Add constraints
    #Interpreter must speak the language
    for b in Blocks:
        for s in Sessions_blocks[b]:
            #change by pairwise language
            for l in Pairwise_languages_sessions[s]:
                l1, l2 = l
                for i in Interpreters:
                    if l1 not in Languages_interpreters[i] or l2 not in Languages_interpreters[i]:
                        model.addConstr(assign[b, s, l, i] == 0, name=f"lang_{b}_{s}_{l}_{i}")
    
    #One interpreter can translate only one session at a time
    for b in Blocks:
        for i in Interpreters:
            model.addConstr(gp.quicksum(assign[b, s, l, i] for s in Sessions_blocks[b] for l in Pairwise_languages_sessions[s]) <= 1, 
                            name=f"interpreter_{b}_{i}")
    
    # Each session can have only one interpreter per language
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l in Pairwise_languages_sessions[s]:
                model.addConstr(gp.quicksum(assign[b, s, l, i] for i in Interpreters) <= 1, 
                                name=f"session_{s}_{l}")
                
    #Session covered if all languages are covered
    for b in Blocks:
        for s in Sessions_blocks[b]:
            model.addConstr(session_covered[s] <= gp.quicksum(assign[b, s, l, i]
                                                            for l in Pairwise_languages_sessions[s]
                                                            for i in Interpreters) / len(Pairwise_languages_sessions[s]), 
                                                            name=f"session_covered_{s}")
            
    #Max 15 hours per interpreter
    for i in Interpreters:
        model.addConstr(gp.quicksum(assign[b, s, l, i] for b in Blocks for s in Sessions_blocks[b] 
                                    for l in Pairwise_languages_sessions[s]) <= 15, 
                        name=f"max_hours_{i}")
    
    # Max 3 sessions in a row per interpreter
    for i in Interpreters:
        for j in range(len(Sessions_blocks) - 3):
                model.addConstr(gp.quicksum(assign[b, s, l, i] for b in Blocks[j:j+4] for s in Sessions_blocks[b] for l in Pairwise_languages_sessions[s]) <= 3, 
                                name=f"max_sessions_{b}_{s}_{i}")

    # Set objective
    #Maximize the number of languages translated
    #OBJ1
    if OBJ_FUNC == 1:
        model.setObjective(gp.quicksum(assign[b, s, l, i] for b in Blocks for s in Sessions_blocks[b] 
                                        for l in Pairwise_languages_sessions[s] for i in Interpreters), GRB.MAXIMIZE)
    #
    #Max the number of sessions translated
    if OBJ_FUNC == 2:
        model.setObjective(gp.quicksum(session_covered[s] for b in Blocks for s in Sessions_blocks[b]), GRB.MAXIMIZE)
    
    return model

In [240]:
#Open jason file to load data
with open(r"C:\Users\thomv\Documents\ULB\MA2\Combinatorial optimization\Project\instances\test1.json", "r") as file:
    data2 = json.load(file)
print(data2["Blocks"])
print(data2["Sessions"])
print(data2["Sessions_b"])

model_2 = Model_2(data2)
model_2.optimize()
# Print the results
for v in model_2.getVars():
    if (v.x == 1):
        print(f"{v.varName}: {v.x}")

['Block 001', 'Block 002', 'Block 003', 'Block 004', 'Block 005']
['Session 001', 'Session 002', 'Session 003', 'Session 004', 'Session 005', 'Session 006', 'Session 007']
{'Block 001': ['Session 001', 'Session 002', 'Session 003'], 'Block 002': ['Session 004'], 'Block 003': ['Session 005'], 'Block 004': ['Session 006'], 'Block 005': ['Session 007']}
Pairwise languages sessions: {'Session 001': [('English', 'French'), ('English', 'Spanish'), ('French', 'Spanish')], 'Session 002': [('German', 'Spanish')], 'Session 003': [('English', 'Spanish')], 'Session 004': [('French', 'Italian')], 'Session 005': [('French', 'Italian')], 'Session 006': [('French', 'Italian')], 'Session 007': [('French', 'Italian')]}
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 93 rows, 52 columns and

In [ ]:
list = [1, 2, 3 , 4, 5]
for i in range(len(list) - 1):
    print(list[i: i + 2])


[1, 2]
[2, 3]
[3, 4]


In [ ]:
def Model_3(data, OBJ_FUNC=1):
    #Bridge implementation
    model = gp.Model("model with bridge")
    
    
    #unpack data
    Blocks = data["Blocks"]
    Sessions = data["Sessions"]
    Sessions_blocks = data["Sessions_b"]
    Interpreters = data["Interpreters"]
    Languages = data["Languages"]
    Languages_sessions = data["Languages_s"]
    Languages_interpreters = data["Languages_i"]

    #Generate the pairwise combinations of languages
    Pairwise_languages_sessions = Languages_sessions.copy()
    for s in Sessions:
        Pairwise_languages_sessions[s] = [(l1, l2) for i, l1 in enumerate(Languages_sessions[s]) 
                                          for l2 in Languages_sessions[s][i+1:]]
    print("Pairwise languages sessions:", Pairwise_languages_sessions)

    # Create variables
    #x[b, s, language, interpreter] = 1 if block b is assigned to session s with language l and interpreter i
    assign = {}
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l in Pairwise_languages_sessions[s]:
                for i in Interpreters:
                    assign[b, s, l, i] = model.addVar(vtype=GRB.BINARY, name=f"x_{b}_{s}_{l}_{i}")
    
    bridge = {}
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l in Pairwise_languages_sessions[s]:
                for i1 in Interpreters:
                    for i2 in Interpreters:
                        if i1 != i2:
                            bridge[b, s, l, i1, i2] = model.addVar(vtype=GRB.BINARY, name=f"bridge_{b}_{s}_{l}_{i1}_{i2}")

    session_covered = {}
    for s in Sessions:
        #session_covered[s] = 1 if session s is covered by at least one interpreter
        session_covered[s] = model.addVar(vtype=GRB.BINARY, name=f"session_covered_{s}")
    
    # Add constraints
    #Interpreter must speak the language
    for b in Blocks:
        for s in Sessions_blocks[b]:
            #change by pairwise language
            for l in Pairwise_languages_sessions[s]:
                l1, l2 = l
                for i in Interpreters:
                    if l1 not in Languages_interpreters[i] or l2 not in Languages_interpreters[i]:
                        model.addConstr(assign[b, s, l, i] == 0, name=f"lang_{b}_{s}_{l}_{i}")
    
    #One interpreter can translate only one session at a time
    for b in Blocks:
        for i in Interpreters:
            model.addConstr(gp.quicksum(assign[b, s, l, i] for s in Sessions_blocks[b] for l in Pairwise_languages_sessions[s]) <= 1, 
                            name=f"interpreter_{b}_{i}")
    
    # Each session can have only one interpreter per language
    for b in Blocks:
        for s in Sessions_blocks[b]:
            for l in Pairwise_languages_sessions[s]:
                model.addConstr(gp.quicksum(assign[b, s, l, i] for i in Interpreters) <= 1, 
                                name=f"session_{s}_{l}")
                
    #Session covered if all languages are covered
    for b in Blocks:
        for s in Sessions_blocks[b]:
            model.addConstr(session_covered[s] <= gp.quicksum(assign[b, s, l, i]
                                                            for l in Pairwise_languages_sessions[s]
                                                            for i in Interpreters) / len(Pairwise_languages_sessions[s]), 
                                                            name=f"session_covered_{s}")
            
    #Max 15 hours per interpreter
    for i in Interpreters:
        model.addConstr(gp.quicksum(assign[b, s, l, i] for b in Blocks for s in Sessions_blocks[b] 
                                    for l in Pairwise_languages_sessions[s]) <= 15, 
                        name=f"max_hours_{i}")
    
    # Max 3 sessions in a row per interpreter
    for i in Interpreters:
        for j in range(len(Sessions_blocks) - 3):
                model.addConstr(gp.quicksum(assign[b, s, l, i] for b in Blocks[j:j+4] for s in Sessions_blocks[b] for l in Pairwise_languages_sessions[s]) <= 3, 
                                name=f"max_sessions_{Blocks[j]}_{s}_{i}")
                


    # Set objective
    #Maximize the number of languages translated
    #OBJ1
    if OBJ_FUNC == 1:
        model.setObjective(gp.quicksum(assign[b, s, l, i] for b in Blocks for s in Sessions_blocks[b] 
                                        for l in Pairwise_languages_sessions[s] for i in Interpreters), GRB.MAXIMIZE)
    #
    #Max the number of sessions translated
    if OBJ_FUNC == 2:
        model.setObjective(gp.quicksum(session_covered[s] for b in Blocks for s in Sessions_blocks[b]), GRB.MAXIMIZE)
    
    return model